In [2]:
from azureml.core import Workspace
import azureml.core

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 0.1.0.1005480


In [3]:
ws = Workspace.get_or_create(name='mladsdemo', location='eastus2', subscription_id='6a275e9b-a3e6-42c2-b158-760d27f5cb83', resource_group='mladsdemo')
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

Workspace name: mladsdemo
Azure region: eastus2
Subscription id: 6a275e9b-a3e6-42c2-b158-760d27f5cb83
Resource group: mladsdemo


In [4]:
# get the text data from the github repo and unzip it
from fit_and_store_pipeline import unzip_file_here
import urllib
import os

if not os.path.isfile('./attack_data.csv'):
    if not os.path.isfile('./text_data.zip'): 
        urllib.request.urlretrieve('https://activelearning.blob.core.windows.net/activelearningdemo/text_data.zip', 'text_data.zip')
    unzip_file_here('text_data.zip')

if not os.path.isfile('miniglove_6B_50d_w2v.txt'):
    unzip_file_here('miniglove_6B_50d_w2v.zip')
    
print('Data files here')

C:\Users\tosingli\AppData\Local\conda\conda\envs\myenv\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Data files here


In [5]:
from fit_and_store_pipeline import *
script_main()

[[845  12]
 [ 95  48]]
{"result": [1, 0, 0, 0, 1]}


In [110]:
%%writefile myenv.yml

# my attempt 1 at env file: packages are in pip
name: myenv
channels:
  - defaults
dependencies:
  - pip:        
    - scikit-learn
    - numpy
    - scipy
    - numexpr
    - gensim
    # Required packages for AzureML execution, history, and data preparation.
    - --extra-index-url https://azuremlsdktestpypi.azureedge.net/sdk-release/Preview/E7501C02541B433786111FE8E140CAA1
    - azureml-core

Overwriting myenv.yml


In [6]:
%%writefile myenv.yml

# give this a try: packages are in conda
name: myenv
channels:
  - defaults
dependencies:
    - scikit-learn
    - gensim

Overwriting myenv.yml


In [7]:
# you may need to register the provider in your subscription 
# az provider register -n Microsoft.ContainerInstance

from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 7, 
                                               tags = ['MLADS 2018 Text Workshop'], 
                                               description = 'Sample container deployment')

In [8]:
import os
os.getcwd()
os.path.isfile('rf_attack_classifier_pipeline.pkl')

True

In [15]:
%%time
# this will take 5-10 minutes to finish
service = ws.deploy_webservice(name = 'aci-attack-clsfr2',
                               deploy_config = aciconfig,
                               target = None,
                               models = ['rf_attack_classifier.pkl'],
                               runtime = 'python',
                               conda_file = 'myenv.yml',
                               # driver = 'fit_and_store_pipeline.py'
                               driver='bare_scoring.py')
service.wait_for_deployment(show_output = True)

Registering model rf_attack_classifier.pkl


Client-Request-ID=7514b702-6e5b-11e8-a54d-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Tue, 12 Jun 2018 16:12:52 GMT, Server-Request-ID=244121ea-801e-015f-4b68-022849000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:244121ea-801e-015f-4b68-022849000000Time:2018-06-12T16:12:52.3542707Z</Message></Error>.
Client-Request-ID=75621718-6e5b-11e8-9a9b-dc53600a4933 Retry policy did not allow for a retry: Server-Timestamp=Tue, 12 Jun 2018 16:12:52 GMT, Server-Request-ID=3094085a-001e-002c-4268-021edf000000, HTTP status code=409, Exception=The specified container already exists.ErrorCode: ContainerAlreadyExists<?xml version="1.0" encoding="utf-8"?><Error><Code>ContainerAlreadyExists</Code><Message>The specified container already exists.RequestId:3094085a-001e-002c-4268-021ed

Creating image
Image creation operation finished for image aci-attack-clsfr2:5, operation "Succeeded"
Creating service
Running..................................................
FailedACI service creation operation finished, operation "Failed"
Service creation failed, unexpected error response:
{'code': 'AciDeploymentFailed', 'message': 'Aci Deployment failed', 'details': [{'code': 'CrashLoopBackOff', 'message': 'CrashLoopBackOff: Back-off 20s restarting failed'}]}
Wall time: 7min 10s


In [72]:
# Let's try to do it the "4 steps" way
# Step 1: Make an image, this usually succeeds
# image = ws.create_image(name, models, runtime, driver_file, docker_file=None, conda_file=None, dependencies=None, schema_file=None, enable_gpu=False, tags=None, description=None)
img = ws.create_image(name = 'aci-attack-clsfr4-img',                               
                      models = ['rf_attack_classifier_pipeline.pkl'],
                      runtime = 'python',
                      conda_file = 'myenv.yml',
                      driver_file = 'fit_and_store_pipeline.py')

KeyboardInterrupt: 

In [74]:
# We can also get the image by name from the Workspace
img = ws.get_image(id="aci-attack-clsfr4:1")

In [82]:
# Let's try to do it the "4 steps" way
# Step 2: Deploy image, this usually times out
from azureml.core.webservice import Webservice

service = Webservice.deploy_from_image(workspace=ws,
                                       name = 'aci-attack-clsfr5-srv',
                                       image=img,
                                       deployment_config = aciconfig)

Creating service


In [85]:
# we can get the service by name
service = ws.get_webservice(name='aci-attack-clsfr4-srv')

In [86]:
# verify the deployed service
print('web service is hosted in ACI:', service.scoring_uri)

web service is hosted in ACI: http://138.91.127.140:5001/score


In [10]:
# get service after timeout
from azureml.core.webservice import Webservice
services = Webservice.list(workspace=ws)
print(services)
print(len(services))

[<azureml.core.webservice.aci.AciWebservice object at 0x00000249E923F5F8>, <azureml.core.webservice.aci.AciWebservice object at 0x00000249E923F6D8>, <azureml.core.webservice.aci.AciWebservice object at 0x00000249E923F0F0>]
3


In [11]:
# delete existing services before deploying new ones
services = Webservice.list(workspace=ws)
for i in range(0, len(services)):
    try:
        services[i].delete()
    except:
        pass

In [12]:
# delete failed images
from azureml.core.image import Image
images = Image.list(workspace=ws)
for i in range(0, len(images)):
    try:
        images[i].delete()
    except:
        pass

In [89]:
# Now score the webservice

import json

test_attacks = [ 'You are scum.', 'I like your shoes.', 'You are pxzx.', 
                 'Your mother was a hamster and your father smelt of elderberries',
                 'Buy some hair slime for me, please']

# score the first row from the test set.
test_samples = bytes(json.dumps({"data": test_attacks}), encoding = 'utf8')
test_samples

b'{"data": ["You are scum.", "I like your shoes.", "You are pxzx.", "Your mother was a hamster and your father smelt of elderberries", "Buy some hair slime for me, please"]}'

In [90]:
service.run(input_data = test_samples)

KeyboardInterrupt: 

In [14]:
service.get_logs()

'2018-06-12 16:09:44,301 CRIT Supervisor running as root (no user in config file)\n2018-06-12 16:09:44,303 INFO supervisord started with pid 1\n2018-06-12 16:09:45,305 INFO spawned: \'rsyslog\' with pid 7\n2018-06-12 16:09:45,307 INFO spawned: \'program_exit\' with pid 8\n2018-06-12 16:09:45,308 INFO spawned: \'nginx\' with pid 9\n2018-06-12 16:09:45,309 INFO spawned: \'iot\' with pid 10\n2018-06-12 16:09:45,311 INFO spawned: \'gunicorn\' with pid 11\n2018-06-12 16:09:45,402 INFO success: iot entered RUNNING state, process has stayed up for > than 0 seconds (startsecs)\nEdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...\n2018-06-12 16:09:45,471 INFO exited: iot (exit status 1; expected)\n2018-06-12 16:09:46,472 INFO success: rsyslog entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-12 16:09:46,472 INFO success: program_exit entered RUNNING state, process has stayed up for > than 1 seconds (startsecs)\n2018-06-12 16:09:50,477 